# Entrenamiento del modelo

In [ ]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Definir transformaciones
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'directorio_datos'  # Actualiza esto con la ruta a tu conjunto de datos
batch_size = 32

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
from torch import nn

# Cargar el modelo preentrenado
model = models.densenet121(pretrained=True)

# Reemplazar la última capa para ajustar al número de clases (5 en este caso)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)


In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 12  # Puedes ajustar el número de épocas

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']

    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')


# Guardado del modelo

In [ ]:
torch.save(model.state_dict(), 'densenet_model1.pth')

# Matriz de confusion

In [ ]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix

# Asegúrate de que el modelo esté en modo de evaluación
model.eval()

# Define las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dispositivo (GPU o CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)# Ruta al conjunto de datos
data_dir = 'directorio_datos_test'

# Cargar el conjunto de datos
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
class_names = dataset.classes

y_pred = []
y_true = []

# Desactivar el cálculo de gradientes
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Calcular la matriz de confusión
confusion_mat = confusion_matrix(y_true, y_pred)
print(confusion_mat)

# Recall

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18  # Asumiendo que estás usando ResNet18 como ejemplo
from sklearn.metrics import recall_score
import numpy as np

# Asegúrate de tener el dispositivo adecuado para PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define las transformaciones para tus datos de prueba
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga tus datos de prueba
test_dataset = datasets.ImageFolder(root='directorio_datos_test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Función para cargar el modelo .pth
def load_model_pth(model_path, num_classes):
    # Cargar el modelo preentrenado
    model = models.densenet121(pretrained=True)

    # Reemplazar la última capa para ajustar al número de clases (5 en este caso)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 5)
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    return model

# Carga el modelo .pth
model_path = '/kaggle/working/densenet_model1.pth'  # Asume que ya has copiado tu modelo al directorio de trabajo
model = load_model_pth(model_path, num_classes=5)  # Ajusta num_classes según tu caso

# Predicción y cálculo de recall
all_preds = []
true_classes = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        true_classes.extend(labels.cpu().numpy())

# Calcula el recall
recall = recall_score(true_classes, all_preds, average='macro')  # Ajusta el average según tu caso

print(f'Recall del modelo: {recall}')
